In [1]:
import pandas as pd 
import numpy as np
import sklearn
import keras
from keras import models
from keras import layers
from keras.layers import LSTM
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score #The F-Measure is a popular metric for imbalanced classification.
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
# import Google Drive 套件
from google.colab import drive

# 將自己的雲端硬碟掛載上去
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# load models
from keras.models import load_model
Intensity_Classifier = load_model('gdrive/My Drive/金融科技Final_project/Intensity_Classifier')
Small_Event_Classifier = load_model('gdrive/My Drive/金融科技Final_project/Small_Event_Classifier')
Big_Event_Classifier = load_model('gdrive/My Drive/金融科技Final_project/Big_Event_Classifier')
Month_abnormal_returns_Classifier = load_model('gdrive/My Drive/金融科技Final_project/Month_abnormal_returns_Classifier')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [4]:
# load encoders
from joblib import dump, load

Intensity_enc = load('gdrive/My Drive/金融科技Final_project/Intensity_enc.joblib')

BE_enc = load('gdrive/My Drive/金融科技Final_project/BE_enc.joblib')

SE_enc = load('gdrive/My Drive/金融科技Final_project/SE_enc.joblib')

In [5]:
"""
# Here we have to use our original text to fit our tokenizer
data_df = pd.read_excel('gdrive/My Drive/金融科技Final_project/2019+2020年新聞資訊事件_股價報酬_528.xlsx')
tokenizer = keras .preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(data_df.content_tokenized)

# Save our tokenizer
from joblib import dump, load
dump(tokenizer, 'gdrive/My Drive/金融科技Final_project/my_tokenizer.joblib') 
"""

# load tokenizer

from joblib import dump, load

my_tokenizer = load('gdrive/My Drive/金融科技Final_project/my_tokenizer.joblib')

In [6]:
# Here is our preprocess_text function

def preprocess_text(Tokenizer,corpus, MAX_SEQUENCE_LENGTH = 1000):
  '''
  put our fitted tokernizer into the "Tokernizer" of the function
  put our cutted text into the "corpus" of the function
  '''
  output = Tokenizer.texts_to_sequences(corpus)
  output = keras .preprocessing .sequence .pad_sequences(output, maxlen=MAX_SEQUENCE_LENGTH)
  return output

In [7]:
def predict_intensity(x):
  pred_Intensity = Intensity_Classifier.predict(x)
  pred_Intensity =(pred_Intensity>0.5).astype(int)

  # decode categorical dummy variables to categorical variables
  pred_Intensity = Intensity_enc.inverse_transform(pred_Intensity)
  return pred_Intensity

In [8]:
def predict_Big_Event(x):
  pred_Big_Event = Big_Event_Classifier.predict(x)
  pred_Big_Event =(pred_Big_Event>0.5).astype(int)

  # decode categorical dummy variables to categorical variables
  pred_Big_Event = BE_enc.inverse_transform(pred_Big_Event)
  return pred_Big_Event

In [9]:
def predict_Small_Event(x):
  pred_Small_Event = Small_Event_Classifier.predict(x)
  pred_Small_Event =(pred_Small_Event>0.5).astype(int)

  # decode categorical dummy variables to categorical variables
  pred_Small_Event = SE_enc.inverse_transform(pred_Small_Event)
  return pred_Small_Event

In [10]:
def predict_Month_abnormal_returns(x):
  pred_Month_abnormal_returns = Month_abnormal_returns_Classifier.predict(x)
  pred_Month_abnormal_returns =(pred_Month_abnormal_returns>0.5).astype(int)
  return pred_Month_abnormal_returns

# Model's summary

In [12]:
Big_Event_Classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

In [13]:
Small_Event_Classifier.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_9 (Dense)              (None, 15)                495       
Total params: 1,333,039
Trainable params: 1,333,039
Non-trainable params: 0
_________________________________________________________________


In [14]:
Intensity_Classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                9280      
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 119       
Total params: 1,289,399
Trainable params: 1,289,399
Non-trainable params: 0
_________________________________________________________________


In [15]:
Month_abnormal_returns_Classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 512)         5120000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                147712    
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 5,271,937
Trainable params: 5,271,937
Non-trainable params: 0
____________________________________________

# Try to use our models

In [16]:
# 透過 gdrive/My Drive/... 來存取檔案
data_df = pd.read_excel('gdrive/My Drive/金融科技Final_project/2019+2020年新聞資訊事件_股價報酬_528.xlsx',sheet_name='year（小事件重分）')

In [17]:
x = data_df.content_tokenized

In [18]:
x

0                                   發言人 林俐婉 內部 調動 由 江巍峰 接任
1                              內部 稽核 主管 林志強 內部 調動 由 莊文清 接任
2                                 會計 主管 藍俊雄 內部 調動 由 林鴻名 接任
3                              內部 稽核 主管 游本詮 內部 調動 由 曾筱茜 接任
4                                 財務 經理 洪廷宜 內部 調動 由 王婷渝 接任
                               ...                        
35043    貨櫃 船運 市場 往年 農曆年 後 二 到 三 周 的 淡季 砍 班 措施 今年 因 新冠 ...
35044    富邦金擬 發行 無 擔保 普通 公司債 富邦金 1091 以 募集 台幣 40000000仟...
35045    國泰金擬 發行 無 擔保 普通 公司 債國 泰金 1091 以 募集 台幣 30000000...
35046    新光 金擬 發行 無 擔保 普通 公司債 新光金 1091 以 募集 台幣 3000000仟...
35047    20200330 TCRI 標 D調 等 因 營收 仍 趨近於 無 公司 轉型 發展 不如 ...
Name: content_tokenized, Length: 35048, dtype: object

In [19]:
x = preprocess_text(Tokenizer = my_tokenizer,corpus = data_df.content_tokenized, MAX_SEQUENCE_LENGTH = 1305)
print(x)

[[   0    0    0 ...  147   31   98]
 [   0    0    0 ...  147   31   98]
 [   0    0    0 ...  147   31   98]
 ...
 [   0    0    0 ...  102  615 1303]
 [   0    0    0 ...    5  198   97]
 [   0    0    0 ...  239  164 2884]]


In [20]:
BE_y = data_df["大事件類別"]
preds_BE = predict_Big_Event(x)

le = sklearn.preprocessing.LabelEncoder()
BE_y = le.fit_transform(BE_y)
preds_BE = le.transform(preds_BE)

accuracy = accuracy_score(BE_y, preds_BE)
precision = precision_score(BE_y, preds_BE,average=None) # average=None: The precision scores for each class are returned
recall = recall_score(BE_y, preds_BE,average=None)
f1 = f1_score(BE_y, preds_BE,average=None)
c_matrix = confusion_matrix(BE_y, preds_BE)
print('accuracy: '+str(accuracy)+'\n')
print('precision: '+str(precision)+'\n')
print('recall: '+str(recall)+'\n')
print('F1 score: '+str(f1)+'\n')
print('Confusion matrix: ')
print(c_matrix)

accuracy: 0.9973179639351746

precision: [0.94691781 0.99888783 0.99772364 0.99873293 1.        ]

recall: [0.99460432 0.99857052 0.99864262 0.99300112 0.99056604]

F1 score: [0.97017544 0.99872915 0.99818292 0.99585878 0.99526066]

Confusion matrix: 
[[  553     0     3     0     0]
 [    4  6287     5     0     0]
 [   18     1 20600     9     0]
 [    8     3    39  7094     0]
 [    1     3     0     0   420]]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
SE_y = data_df["小事件類別"]
preds_SE = predict_Small_Event(x)

le = sklearn.preprocessing.LabelEncoder()
SE_y = le.fit_transform(SE_y)
preds_SE = le.transform(preds_SE)

accuracy = accuracy_score(SE_y, preds_SE)
precision = precision_score(SE_y, preds_SE,average=None) # average=None: The precision scores for each class are returned
recall = recall_score(SE_y, preds_SE,average=None)
f1 = f1_score(SE_y, preds_SE,average=None)
c_matrix = confusion_matrix(SE_y, preds_SE)
print('accuracy: '+str(accuracy)+'\n')
print('precision: '+str(precision)+'\n')
print('recall: '+str(recall)+'\n')
print('F1 score: '+str(f1)+'\n')
print('Confusion matrix: ')
print(c_matrix)

In [21]:
Intensity_y = data_df["事件強度"]
preds_Intensity = predict_intensity(x)

accuracy = accuracy_score(Intensity_y, preds_Intensity)
precision = precision_score(Intensity_y, preds_Intensity,average=None) # average=None: The precision scores for each class are returned
recall = recall_score(Intensity_y, preds_Intensity,average=None)
f1 = f1_score(Intensity_y, preds_Intensity,average=None)
c_matrix = confusion_matrix(Intensity_y, preds_Intensity)
print('accuracy: '+str(accuracy)+'\n')
print('precision: '+str(precision)+'\n')
print('recall: '+str(recall)+'\n')
print('F1 score: '+str(f1)+'\n')
print('Confusion matrix: ')
print(c_matrix)

accuracy: 0.9663318876968728

precision: [0.44420131 0.90369393 0.99049959 0.95615681 0.95558959 0.96842105
 1.        ]

recall: [0.99509804 0.99563953 0.97010695 0.95464853 0.96950864 1.
 1.        ]

F1 score: [0.61422088 0.94744122 0.98019722 0.95540207 0.96249879 0.98395722
 1.        ]

Confusion matrix: 
[[  203     1     0     0     0     0     0]
 [    1   685     0     2     0     0     0]
 [  140    46 18141   322    49     2     0]
 [   85    24   167  9683   183     1     0]
 [   28     2     7   120  4992     0     0]
 [    0     0     0     0     0    92     0]
 [    0     0     0     0     0     0    72]]


In [23]:
Abnormal_return_y = data_df["前5 ~ 25日(月平均)兩日異常報酬"]
preds_Abnormal_return = predict_Month_abnormal_returns(x)

In [24]:
Abnormal_return_y

0       -3.614215
1       -0.203923
2       -1.005807
3        0.497570
4       -0.557661
           ...   
35043    1.368266
35044    0.253422
35045    0.345016
35046   -0.778734
35047    1.927342
Name: 前5 ~ 25日(月平均)兩日異常報酬, Length: 35048, dtype: float64

In [25]:
preds_Abnormal_return

array([[0],
       [0],
       [1],
       ...,
       [1],
       [0],
       [1]])

In [ ]:
accuracy = accuracy_score(Abnormal_return_y, preds_Abnormal_return)
precision = precision_score(Abnormal_return_y, preds_Abnormal_return,average=None) # average=None: The precision scores for each class are returned
recall = recall_score(Abnormal_return_y, preds_Abnormal_return,average=None)
f1 = f1_score(Abnormal_return_y, preds_Abnormal_return,average=None)
c_matrix = confusion_matrix(Abnormal_return_y, preds_Abnormal_return)
print('accuracy: '+str(accuracy)+'\n')
print('precision: '+str(precision)+'\n')
print('recall: '+str(recall)+'\n')
print('F1 score: '+str(f1)+'\n')
print('Confusion matrix: ')
print(c_matrix)

# Try encoders

In [ ]:
BE = data_df['大事件類別']
print(BE)

BE = np.asarray(BE)

In [ ]:
BE = BE_enc.transform(BE.reshape(-1, 1))
BE = BE_enc.inverse_transform(BE)

print(BE)

[['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']
 ['M_經營層']]


In [ ]:
SE = data_df['小事件類別']
print(SE)

SE = np.asarray(SE)

0     MT06_高管異動
1     MT06_高管異動
2     MT06_高管異動
3     MT06_高管異動
4     MT06_高管異動
5     MT06_高管異動
6     MT06_高管異動
7     MT02_董監異動
8     MT02_董監異動
9     MT06_高管異動
10    MT06_高管異動
11       經營層_其他
12    MT02_董監異動
13    MT06_高管異動
14    MT06_高管異動
15    MT06_高管異動
16    MT06_高管異動
17    MT06_高管異動
18    MT06_高管異動
19    MT06_高管異動
Name: 小事件類別, dtype: object


In [ ]:
SE = SE_enc.transform(SE.reshape(-1, 1))
SE = SE_enc.inverse_transform(SE)

print(SE)

[['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT02_董監異動']
 ['MT02_董監異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['經營層_其他']
 ['MT02_董監異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']
 ['MT06_高管異動']]


In [ ]:
Intensity = data_df['事件強度']
print(Intensity)

Intensity = np.asarray(Intensity)

0     0
1     0
2     0
3     0
4     0
5     0
6    -1
7     0
8    -1
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: 事件強度, dtype: int64


In [ ]:
Intensity = Intensity_enc.transform(Intensity.reshape(-1, 1))
Intensity = Intensity_enc.inverse_transform(Intensity)

print(Intensity)

[[ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [-1]
 [ 0]
 [-1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]]
